In [1]:
%ls

train_s4_20220311_v5.csv*          uer_churn_feature_selection.py
uer_churn_feature_selection.ipynb


In [2]:
import pandas as pd
import os

In [3]:
pd.set_option('max_columns', None)  # 打印所有列
pd.set_option('max_rows', None)  # 打印所有行

In [12]:
data = pd.read_csv("train_s4_20220311_v5.csv", sep="|")

/home/liuhaozhe/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (162,164,165,166,167,169,170,219,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
data.head()

,uid,register_os,register_country,register_time,life_time,first_pay_time,last_login,version,is_latest_version,last_bankrupt,is_pay,recency,pay_amt,pay_cnt,bankrupt_cnt,first_rank_time,device_brand,device,os,country,is_visitor,grade,vip_lvl,battlepass_lvl,cash_grade,cash_subgrade,rank_grade,rank_subgrade,curr_asset,curr_diamond,product_id,is_bought,show_timestamp,login_cnt_3d,login_cnt_7d,login_cnt_15d,login_cnt_30d,login_days_cnt_3d,login_days_cnt_7d,login_days_cnt_15d,login_days_cnt_30d,login_cnt_per_login_day_30d,pay_cnt_3d,pay_cnt_7d,pay_cnt_15d,pay_cnt_30d,pay_3d,pay_7d,pay_15d,pay_30d,piggy_bank_pay_cnt_3d,piggy_bank_pay_cnt_7d,piggy_bank_pay_cnt_15d,piggy_bank_pay_cnt_30d,piggy_bank_pay_3d,piggy_bank_pay_7d,piggy_bank_pay_15d,piggy_bank_pay_30d,bankrupt_pay_cnt_3d,bankrupt_pay_cnt_7d,bankrupt_pay_cnt_15d,bankrupt_pay_cnt_30d,bankrupt_pay_3d,bankrupt_pay_7d,bankrupt_pay_15d,bankrupt_pay_30d,convinent_pay_cnt_3d,convinent_pay_cnt_7d,convinent_pay_cnt_15d,convinent_pay_cnt_30d,convinent_pay_3d,convinent_pay_7d,convinent_pay_15d,convinent_pay_30d,three_stage_bundle_pay_cnt_3d,three_stage_bundle_pay_cnt_7d,three_stage_bundle_pay_cnt_15d,three_stage_bundle_pay_cnt_30d,three_stage_bundle_pay_3d,three_stage_bundle_pay_7d,three_stage_bundle_pay_15d,three_stage_bundle_pay_30d,lottery_pay_cnt_3d,lottery_pay_cnt_7d,lottery_pay_cnt_15d,lottery_pay_cnt_30d,lottery_pay_3d,lottery_pay_7d,lottery_pay_15d,lottery_pay_30d,mall_pay_cnt_3d,mall_pay_cnt_7d,mall_pay_cnt_15d,mall_pay_cnt_30d,mall_pay_3d,mall_pay_7d,mall_pay_15d,mall_pay_30d,period_bundle_pay_cnt_3d,period_bundle_pay_cnt_7d,period_bundle_pay_cnt_15d,period_bundle_pay_cnt_30d,period_bundle_pay_3d,period_bundle_pay_7d,period_bundle_pay_15d,period_bundle_pay_30d,tier_bundle_pay_cnt_3d,tier_bundle_pay_cnt_7d,tier_bundle_pay_cnt_15d,tier_bundle_pay_cnt_30d,tier_bundle_pay_3d,tier_bundle_pay_7d,tier_bundle_pay_15d,tier_bundle_pay_30d,is_battle_pass_vip,is_newbie_bundle_buyer_30d,pay_mean_recent3,ads_watch_cnt_1d,ads_watch_cnt_3d,ads_watch_cnt_7d,ads_watch_cnt_15d,ads_watch_cnt_30d,ads_click_cnt_1d,mall_ads_watch_cnt,hall_ads_wheel_watch_cnt,bankrupt_popup_watch_cnt,battle_pass_ads_watch_cnt,chip_gain_7d,chip_consume_7d,chip_consume_rate_7d,diamond_gain_7d,diamond_consume_7d,diamond_consume_rate_7d,bandit_gain_7d,bandit_consume_7d,bandit_cnt_3d,bandit_cnt_7d,bandit_cnt_15d,bandit_cnt_30d,all_in_rate_3d,all_in_rate_7d,all_in_rate_15d,all_in_rate_30d,check_rate_3d,check_rate_7d,check_rate_15d,check_rate_30d,fold_rate_3d,fold_rate_7d,fold_rate_15d,fold_rate_30d,raise_rate_3d,raise_rate_7d,raise_rate_15d,raise_rate_30d,last_pay_amount,last_pay_timestamp,last_pay_weekday,last_piggy_bank_bundle_id,last_piggy_bank_pay_amount,last_piggy_bank_pay_timestamp,last_piggy_bank_pay_weekday,pay_piggy_bank_bundle_id_list_30d,pay_piggy_bank_bundle_id_time_dict_30d,pay_piggy_bank_bundle_time_list_recent5_180d,pay_piggy_bank_bundle_time_list_30d,pay_piggy_bank_bundle_time_interval_list_recent5_180d,pay_piggy_bank_bundle_time_interval_list_30d,pay_amt_list_recent5_180d,pay_piggy_bank_bundle_amt_list_recent5_180d,pay_piggy_bank_bundle_amt_list_30d,register_country_arpu,arpu_country,login_day_cnt,last_season_gap_day,max_cash_grade,max_chip_stock,max_blind,bankrupt_cnt_3d,click_unpay_3d,click_unpay_7d,click_unpay_15d,click_unpay_30d,pay_per_day,online_time_per_login_7d,bandit_cnt_1d,bandit_max_1d,time_zone,last_login_time_hour,last_login_gap_day,is_return_player,last_pay_bundle_id,last_pay_bundle_type,pay_bundle_id_list_30d,pay_time_interval_mean_180d,pay_interval_last,pay_amt_list_30d,ads_watch_day_mean_3d,ads_watch_day_mean_7d,ads_watch_day_mean_15d,ads_watch_day_mean_30d,win_per100,active_days_180d,cash_cnt_180d,active_rate_3d,active_rate_7d,active_rate_15d,active_rate_30d,cash_cnt_mean_3d,cash_cnt_mean_7d,cash_cnt_mean_15d,cash_cnt_mean_30d,recommend_piggy_bank_bundle_id_cnt_dict_30d,recommend_unpay_piggy_bank_bundle_id_cnt_dict_30d,recommend_piggy_bank_bundle_id_time_dict_30d,recommend_piggy_bank_bundle_

In [13]:
data.dtypes

uid                                                        int64
register_os                                               object
register_country                                          object
register_time                                             object
life_time                                                  int64
first_pay_time                                            object
last_login                                                object
version                                                   object
is_latest_version                                          int64
last_bankrupt                                             object
is_pay                                                     int64
recency                                                    int64
pay_amt                                                    int64
pay_cnt                                                    int64
bankrupt_cnt                                               int64
first_rank_time          

In [14]:
len(data)

522211

# 去掉缺失数据记录

In [18]:
data = data.dropna(subset=["uid", "is_churned"], axis=0)

In [27]:
len(data)

522211

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [29]:
# train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels="is_churned", axis=1), data["is_churned"], test_size=0.3, random_state=0)
X_train.shape, X_test.shape

((365547, 267), (156664, 267))

In [30]:
X_train_original = X_train.copy()
X_test_original = X_test.copy()

# 转换或丢弃非数值特征

In [40]:
non_numeric_feats = [feat for feat in data.columns if data.dtypes[feat] not in ["int64", "float64"]]
non_numeric_feats

['register_os',
 'register_country',
 'register_time',
 'first_pay_time',
 'last_login',
 'version',
 'last_bankrupt',
 'first_rank_time',
 'device_brand',
 'os',
 'country',
 'pay_piggy_bank_bundle_id_list_30d',
 'pay_piggy_bank_bundle_id_time_dict_30d',
 'pay_piggy_bank_bundle_time_list_recent5_180d',
 'pay_piggy_bank_bundle_time_list_30d',
 'pay_piggy_bank_bundle_time_interval_list_recent5_180d',
 'pay_piggy_bank_bundle_time_interval_list_30d',
 'pay_amt_list_recent5_180d',
 'pay_piggy_bank_bundle_amt_list_recent5_180d',
 'pay_piggy_bank_bundle_amt_list_30d',
 'pay_bundle_id_list_30d',
 'pay_amt_list_30d',
 'recommend_piggy_bank_bundle_id_cnt_dict_30d',
 'recommend_unpay_piggy_bank_bundle_id_cnt_dict_30d',
 'recommend_piggy_bank_bundle_id_time_dict_30d',
 'recommend_piggy_bank_bundle_id_list_recent10_90d',
 'pay_bankrupt_bundle_id_list_30d',
 'recommend_unpay_period_bundle_id_cnt_dict_30d',
 'recommend_unpay_three_stage_bundle_id_cnt_dict_30d',
 'recommend_unpay_lottery_bundle_id_cn

In [41]:
data[non_numeric_feats].head()

,register_os,register_country,register_time,first_pay_time,last_login,version,last_bankrupt,first_rank_time,device_brand,os,country,pay_piggy_bank_bundle_id_list_30d,pay_piggy_bank_bundle_id_time_dict_30d,pay_piggy_bank_bundle_time_list_recent5_180d,pay_piggy_bank_bundle_time_list_30d,pay_piggy_bank_bundle_time_interval_list_recent5_180d,pay_piggy_bank_bundle_time_interval_list_30d,pay_amt_list_recent5_180d,pay_piggy_bank_bundle_amt_list_recent5_180d,pay_piggy_bank_bundle_amt_list_30d,pay_bundle_id_list_30d,pay_amt_list_30d,recommend_piggy_bank_bundle_id_cnt_dict_30d,recommend_unpay_piggy_bank_bundle_id_cnt_dict_30d,recommend_piggy_bank_bundle_id_time_dict_30d,recommend_piggy_bank_bundle_id_list_recent10_90d,pay_bankrupt_bundle_id_list_30d,recommend_unpay_period_bundle_id_cnt_dict_30d,recommend_unpay_three_stage_bundle_id_cnt_dict_30d,recommend_unpay_lottery_bundle_id_cnt_dict_30d,recommend_unpay_bankrupt_bundle_id_cnt_dict_30d,pay_bankrupt_id_time_dict_30d,pay_period_id_time_dict_30d,pay_three_stage_id_time_dict_30d,pay_lottery_id_time_dict_30d,recommend_bankrupt_bundle_id_list_recent10_90d,recommend_period_id_list_recent10_90d,recommend_three_stage_bundle_id_list_recent10_90d,recommend_lottery_bundle_id_list_recent10_90d,pay_bankrupt_bundle_time_list_30d,pay_bankrupt_bundle_time_interval_list_30d,pay_period_bundle_time_interval_list_30d,pay_three_stage_bundle_time_interval_list_30d,pay_lottery_bundle_time_interval_list_30d,pay_bankrupt_bundle_amt_list_recent5_180d,pay_bankrupt_bundle_amt_list_30d,pay_period_bundle_amt_list_30d,pay_three_stage_bundle_amt_list_30d,pay_lottery_bundle_amt_list_30d,pay_period_bundle_id_list_30d,pay_three_stage_bundle_id_list_30d,pay_lottery_bundle_id_list_30d,login_time,login_time_hour_list_7d,login_time_hour_list_30d,login_time_list_7d,login_time_list_30d
0,ios,US,2021-07-15 19:39:02,2021-09-29 21:01:13,2022-03-10 20:14:34,1.5.1,2022-03-10 20:21:21,2021-07-18 23:07:05,15.3.1,ios,US,NaN,{},"1638833500,1639010735,1639011417,1643676163,16...",NaN,"2,2,1,3,1",NaN,"299,299,299,299,299","499,499,499,299,299",NaN,"39,19,19,19,19,19,19,19,19,19","499,299,299,299,299,299,299,299,299,299","{55:2,56:1,54:3}","{55:2,56:1,54:3}","{55:1644542927,54:1644758847,54:1645227486,54:...","54,54,54,55,55,54,54,54,55,56","19,19,19,19,19,19,19,19,19","{112:13,113:2,114:2}","{832:1,739:1,828:1}","{341:2,342:2,343:2,742:1,743:1,348:1,744:1,349...",{19:10},"{19:1644622824,19:1644948144,19:1645391919,19:...",{},{},{},"19,19,19,19,19,19,19,19,19,19","112,112,112,112,112,112,112,112,112,112","739,826,392,740,491,740,832,739,828","350,742,743,744,341,342,343,341,342,343","1644622824,1644948144,1645391919,1645407891,16...","4,4,6,1,5,8,3,4,1",NaN,NaN,NaN,"299,299,299,299,299","299,299,299,299,299,299,299,299,299",NaN,NaN,NaN,NaN,NaN,NaN,2022-03-11 17:56:03,"1,1,1,1,1,1,1","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","2022-03-04 20:42:29,2022-03-06 20:44:39,2022-0...","2022-02-09 18:05:01,2022-02-10 20:28:46,2022-0..."
1,ios,US,2021-07-15 19:39:02,2021-09-29 21:01:13,2022-03-10 20:14:34,1.5.1,2022-03-10 20:21:21,2021-07-18 23:07:05,15.3.1,ios,US,NaN,{},"1638833500,1639010735,1639011417,1643676163,16...",NaN,"2,2,1,3,1",NaN,"299,299,299,299,299","499,499,499,299,299",NaN,"39,19,19,19,19,19,19,19,19,19","499,299,299,299,299,299,299,299,299,299","{55:2,56:1,54:3}","{55:2,56:1,54:3}","{55:1644542927,54:1644758847,54:1645227486,54:...","54,54,54,55,55,54,54,54,55,56","19,19,19,19,19,19,19,19,19","{112:13,113:2,114:2}","{832:1,739:1,828:1}","{341:2,342:2,343:2,742:1,743:1,348:1,744:1,349...",{19:10},"{19:1644622824,19:1644948144,19:1645391919,19:...",{},{},{},"19,19,19,19,19,19,19,19,19,19","112,112,112,112,112,112,112,112,112,112","739,826,392,740,491,740,832,739,828","350,742,743,744,341,342,343,341,342,343","1644622824,1644948144,1645391919,1645407891,16...","4,4,6,1,5,8,3,4,1",NaN,NaN,NaN,"299,299,299,299,299","299,299,299,299,299,299,299,299,299",NaN,NaN,NaN,NaN,NaN,NaN,2022-03-11 17:56:03,"1,1,1,1,1,1,1","1,1,1,1,1,1,1,1

In [42]:
dropped_feats = [ 'pay_piggy_bank_bundle_id_list_30d',
 'pay_piggy_bank_bundle_id_time_dict_30d',
 'pay_piggy_bank_bundle_time_list_recent5_180d',
 'pay_piggy_bank_bundle_time_list_30d',
 'pay_piggy_bank_bundle_time_interval_list_recent5_180d',
 'pay_piggy_bank_bundle_time_interval_list_30d',
 'pay_amt_list_recent5_180d',
 'pay_piggy_bank_bundle_amt_list_recent5_180d',
 'pay_piggy_bank_bundle_amt_list_30d',
 'pay_bundle_id_list_30d',
 'pay_amt_list_30d',
 'recommend_piggy_bank_bundle_id_cnt_dict_30d',
 'recommend_unpay_piggy_bank_bundle_id_cnt_dict_30d',
 'recommend_piggy_bank_bundle_id_time_dict_30d',
 'recommend_piggy_bank_bundle_id_list_recent10_90d',
 'pay_bankrupt_bundle_id_list_30d',
 'recommend_unpay_period_bundle_id_cnt_dict_30d',
 'recommend_unpay_three_stage_bundle_id_cnt_dict_30d',
 'recommend_unpay_lottery_bundle_id_cnt_dict_30d',
 'recommend_unpay_bankrupt_bundle_id_cnt_dict_30d',
 'pay_bankrupt_id_time_dict_30d',
 'pay_period_id_time_dict_30d',
 'pay_three_stage_id_time_dict_30d',
 'pay_lottery_id_time_dict_30d',
 'recommend_bankrupt_bundle_id_list_recent10_90d',
 'recommend_period_id_list_recent10_90d',
 'recommend_three_stage_bundle_id_list_recent10_90d',
 'recommend_lottery_bundle_id_list_recent10_90d',
 'pay_bankrupt_bundle_time_list_30d',
 'pay_bankrupt_bundle_time_interval_list_30d',
 'pay_period_bundle_time_interval_list_30d',
 'pay_three_stage_bundle_time_interval_list_30d',
 'pay_lottery_bundle_time_interval_list_30d',
 'pay_bankrupt_bundle_amt_list_recent5_180d',
 'pay_bankrupt_bundle_amt_list_30d',
 'pay_period_bundle_amt_list_30d',
 'pay_three_stage_bundle_amt_list_30d',
 'pay_lottery_bundle_amt_list_30d',
 'pay_period_bundle_id_list_30d',
 'pay_three_stage_bundle_id_list_30d',
 'pay_lottery_bundle_id_list_30d']

In [43]:
non_numeric_feats = [feat for feat in non_numeric_feats if feat not in dropped_feats]
non_numeric_feats

['register_os',
 'register_country',
 'register_time',
 'first_pay_time',
 'last_login',
 'version',
 'last_bankrupt',
 'first_rank_time',
 'device_brand',
 'os',
 'country',
 'login_time',
 'login_time_hour_list_7d',
 'login_time_hour_list_30d',
 'login_time_list_7d',
 'login_time_list_30d']

# 去掉常量特征和变化极小的特征

In [44]:
const_features = [feat for feat in X_train.columns if X_train[feat].std() == 0]

TypeError: could not convert string to float: 'ios'